In [1]:
from keras.models import Sequential, Model
from keras.applications import VGG16
from keras.layers import (Dense, BatchNormalization, Convolution2D, 
                         MaxPooling2D, Dropout, Input, Flatten)
from keras.optimizers import Adam
from keras_utilities import Vgg16, get_batches, onehot
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5103)


In [2]:
vgg = Vgg16()

In [3]:
# path = "sample/"
# path = "data/"
path = "microsample/"

In [4]:
trn_batches = get_batches(path+"train")
val_batches = get_batches(path+"valid")

Found 2057 images belonging to 10 classes.
Found 1983 images belonging to 10 classes.


In [5]:
trn_labels = onehot(trn_batches.classes)
val_labels = onehot(val_batches.classes)

Fine-tune the vgg model.

In [6]:
vgg.ft(10)

Fit a baseline model to see what happens without any data augmentation.

In [7]:
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.3335 - acc: 0.2912 - val_loss: 2.9975 - val_acc: 0.3469


Without any augmentation, we get a val loss of 2.9975.

## ZCA Whitening

In [8]:
gen = ImageDataGenerator(zca_whitening=True)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
   4/2057 [..............................] - ETA: 93s - loss: 1.7248 - acc: 0.5000

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py:482: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


2057/2057 [==============================] - 94s - loss: 3.4494 - acc: 0.2820 - val_loss: 2.1387 - val_acc: 0.3853


Substantial improvement to 2.1387.

**Conclusion: use ZCA whitening**

## Rotation Range

In [12]:
gen = ImageDataGenerator(rotation_range=.1)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.2577 - acc: 0.3009 - val_loss: 2.9101 - val_acc: 0.3278


Helped a little, not much.

In [13]:
gen = ImageDataGenerator(rotation_range=.2)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.4038 - acc: 0.2839 - val_loss: 2.1824 - val_acc: 0.4433


Nice! Solid val loss.

In [15]:
gen = ImageDataGenerator(rotation_range=.2)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)


Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.2890 - acc: 0.3004 - val_loss: 3.0651 - val_acc: 0.3495


**Conclusion: rotation range of 0.2. **

## Width Shift Range

In [20]:
gen = ImageDataGenerator(width_shift_range=.1)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.4075 - acc: 0.2708 - val_loss: 2.7971 - val_acc: 0.3298


Better than without.

In [21]:
gen = ImageDataGenerator(width_shift_range=.2)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.6077 - acc: 0.2280 - val_loss: 2.0524 - val_acc: 0.4175


WHOA big improvement!

In [22]:
gen = ImageDataGenerator(width_shift_range=.3)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.5783 - acc: 0.2436 - val_loss: 2.5521 - val_acc: 0.3807


Worse

** Conclusion: 0.2 width shift range. **

## Height Shift Range

In [23]:
gen = ImageDataGenerator(height_shift_range=.1)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.4127 - acc: 0.2640 - val_loss: 2.6254 - val_acc: 0.2925


In [24]:
gen = ImageDataGenerator(height_shift_range=.2)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.4198 - acc: 0.2533 - val_loss: 2.6658 - val_acc: 0.3676


Worse

In [25]:
gen = ImageDataGenerator(height_shift_range=.3)
trn_batches = get_batches(path+"train", gen=gen)
vgg = Vgg16()
vgg.ft(10)
vgg.model.fit_generator(generator=trn_batches, samples_per_epoch=trn_batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Found 2057 images belonging to 10 classes.
Epoch 1/1
2057/2057 [==============================] - 94s - loss: 3.4300 - acc: 0.2640 - val_loss: 2.4316 - val_acc: 0.3585
